In [1]:
!pip install pandas xlrd ipdb

In [2]:
import os
import pandas as pd
import ipdb
from datetime import datetime, time

In [3]:
%ls data

weekly-2020-02-10 0925.xlsx


In [4]:
libs = sorted(pd.read_csv('libraries.csv').full_name.tolist())

In [5]:
# CONVERT
dn = 'data'
fn = dn + '/' + 'weekly-2020-02-10 0925.xlsx'
xl = pd.ExcelFile(fn)
sheets = xl.sheet_names
res=[]
for sheet in sheets:
    df = xl.parse(sheet)
    date = datetime.strptime(df.columns[0], '%d %B %Y')
    df = (
        df.loc[df['Counter Name'].isin(libs)]
        .rename(columns={df.columns[0]: 'dt'})
        .fillna('x')        
    )
    times=df.loc[df.dt!='x']
    df.dt = df.dt.replace(to_replace='x', method='ffill')
    dt = list(map(lambda x: datetime.combine(date, time(int(x[:2]), 0)), df.dt.tolist()))
    df.dt = dt
    res.append(df)                             

df = pd.concat(res)
df = df.rename(columns={'Counter Name': 'Library'})

In [6]:
def to_hours(x):
    return pd.to_datetime(x).hour

oh = pd.read_csv('opening hours.csv', converters={'start': lambda x: to_hours(x), 'finish': lambda x: to_hours(x)})
oh

,library,day,start,finish,opening_type
0,Burnt Oak,Mon,14,20,staffed
1,Burnt Oak,Wed,9,12,staffed
2,Burnt Oak,Fri,14,17,staffed
3,Burnt Oak,Sat,14,17,staffed
4,Burnt Oak,Mon,9,14,SSO
...,...,...,...,...,...
144,North Finchley,Thu,12,20,SSO
145,North Finchley,Fri,17,20,SSO
146,East Finchley,Thu,17,20,SSO
147,Osidge,Thu,17,20,SSO


In [7]:
check = df.iloc[246][['dt', 'Library']]
def get_ot(dt, lib):
    in_df = pd.concat([dt, lib], axis=1).reset_index()
    import ipdb
    ipdb.set_trace()
    day = dt.day_name()[:3]
    mask = (
        (oh.library==lib) &
        (oh.day==day) &
        (oh.start<=dt.hour) &
        (oh.finish>dt.hour)
    )
    ot = oh.loc[mask].opening_type.values[0]
    return ot

df.assign(opening_type=get_ot(df['dt'], df.Library))
df

> <ipython-input-7-a37427e9500f>(5)get_ot()
      4     ipdb.set_trace()
----> 5     day = dt.day_name()[:3]
      6     mask = (

ipdb> pd.DataFrame(dt, lib)
*** ValueError: cannot reindex from a duplicate axis
ipdb> pd.merge(dt, lib)
*** pandas.errors.MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False
ipdb> pd.concat([dt, lib], axis=1).reset_index()
      index                  dt          Library
0         0 2020-02-03 00:00:00  Chipping Barnet
1         6 2020-02-03 00:00:00        Colindale
2         9 2020-02-03 00:00:00    East Finchley
3        12 2020-02-03 00:00:00          Edgware
4        15 2020-02-03 00:00:00    Golders Green
...     ...                 ...              ...
1171    561 2020-02-09 23:00:00    East Finchley
1172    564 2020-02-09 23:00:00          Edgware
1173    567 2020-02-09 23:00:00    Golders Green
1174    570 2020-02-09 23:00:00   North Finchley
1175    573 2020-02-09 23:0

BdbQuit: 